In [5]:
import os
import glob
import sys
import numpy as np
import pickle
import tensorflow as tf
import PIL
import ipywidgets
import io

""" make sure this notebook is running from root directory """
while os.path.basename(os.getcwd()) in ('notebooks', 'src'):
    os.chdir('..')
assert ('README.md' in os.listdir('./')), 'Can not find project root, please cd to project root before running the following code'

import src.tl_gan.generate_image as generate_image
import src.tl_gan.feature_axis as feature_axis
import src.tl_gan.feature_celeba_organize as feature_celeba_organize
from src.notebooks.util import get_feature_data as get_feature_data


"""
Takes in a 40-D description of an individual, converts this into a 512-D space
(using 'feature_direction', and returns an image of the predicted indvidual.

@param  features: a list of 40 values
returns: an image as a result of the prediction model
"""
def feature_to_image(features, save_name=None):
    # Create tf session
    yn_CPU_only = False
    if yn_CPU_only:
        config = tf.ConfigProto(device_count = {'GPU': 0}, allow_soft_placement=True)
    else:
        config = tf.ConfigProto(allow_soft_placement=True)
        config.gpu_options.allow_growth = True

    sess = tf.InteractiveSession(config=config)
    
    feature_direction, feature_name, num_feature = get_feature_data()
    
    feature_lock_status = np.zeros(len(feature_direction)).astype('bool')
    feature_directoion_disentangled = feature_axis.disentangle_feature_axis_by_idx(
        feature_direction, idx_base=np.flatnonzero(feature_lock_status))
    
    z_sample = np.random.randn(512)
    feature_direction_transposed = np.transpose(feature_direction)

    step_size = 5
    
    for direction, feature_val, idx_feature in zip(feature_direction_transposed, features, range(len(features))):
        z_sample = np.add(z_sample, feature_val * feature_directoion_disentangled[:, idx_feature] * step_size)
    
    path_model = './asset_model/karras2018iclr-celebahq-1024x1024.pkl'
    try:
        with open(path_model, 'rb') as file:
            G, D, Gs = pickle.load(file)
    except FileNotFoundError:
        print('before running the code, download pre-trained model to project_root/asset_model/')
        raise
    
    x_sample = generate_image.gen_single_img(z=z_sample, Gs=Gs)
    
#     if save_name != None:
    generate_image.save_img(x_sample, save_name)
#     else:
#         pass # TODO: flask interaction

In [6]:
"""
Helper method to take in a feature dictionary that is partially filled, and generate a prediction image from it.
"""
def dict_to_image(feature_dict):
    # Load feature directions and labels
    path_feature_direction = './asset_results/pg_gan_celeba_feature_direction_40'

    pathfile_feature_direction = glob.glob(os.path.join(path_feature_direction, 'feature_direction_*.pkl'))[-1]

    with open(pathfile_feature_direction, 'rb') as f:
        feature_direction_name = pickle.load(f)

    feature_names = feature_direction_name['name']
    
    print(feature_names)

    features = []
    
    for feature_name in feature_names:
        if feature_name in feature_dict.keys():
            features.append(feature_dict.get(feature_name))
        else:# feature_name in default_feature_dict:
            features.append(0)

    feature_to_image(features)

In [7]:
# Sample method call
for i in range(0, 20, 2):
    test = [(i - 10) * 0.2] + [-0.4] * 39
    feature_to_image(test, "src/notebooks/out/yeeezzzzzzzz{}.jpg".format(i))
    

test = [-9] + [-0.4] * 39
feature_to_image(test, "src/notebooks/out/meh{}.jpg".format(999))

InternalError: CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory